# Basic Usage

## What's in this notebook?
This notebook demonstrates the basic usage of the ParabolaRegression class. The ParabolaRegression class fits a parabolic function to the data, capturing the quadratic relationship between the independent and dependent variables. This model is useful for identifying non-linear patterns in the data and can be applied to various domains, including economics, psychophysics, and decision-making under uncertainty.

In this notebook, we will:
1. Import the required libraries.
2. Define a benchmark function to evaluate the ParabolaRegression class on synthetic experiments.
3. Demonstrate the ParabolaRegression class's performance on three synthetic experiments:
4. Steven's Power Law: A psychophysical law describing the relationship between stimulus intensity and perceived magnitude.
5. Weber-Fechner Law: A psychophysical law quantifying the minimum change in a stimulus required to be noticeable.
6. Expected Utility Model with Two Choice Options: A decision-making model evaluating the expected value of two options under uncertainty.
7. Evaluate the ParabolaRegression class's performance on each experiment, including fitting the model, predicting the validation set, and obtaining the identified equation.

Let's get started!

## Installation

In [ ]:
%%capture
!pip install autora
!pip install autora[all-theorists]

## Import Required Libraries

In [ ]:
from autora.experiment_runner.synthetic.economics.expected_value_theory import expected_value_theory
from autora.experiment_runner.synthetic.psychophysics.stevens_power_law import stevens_power_law
from autora.experiment_runner.synthetic.psychophysics.weber_fechner_law import weber_fechner_law
from sklearn.model_selection import train_test_split

from autora.theorist.autora_theorist_avengers import ParabolaRegression

## Define the benchmark function

In [ ]:
def benchmark(experiment_runner, theorist):
    # generate all conditions
    conditions = experiment_runner.domain()

    # generate all corresponding observations
    experiment_data = experiment_runner.run(conditions, added_noise=0.01)

    # get the name of the independent and independent variables
    ivs = [iv.name for iv in experiment_runner.variables.independent_variables]
    dvs = [dv.name for dv in experiment_runner.variables.dependent_variables]

    # extract the dependent variable (observations) from experiment data
    conditions = experiment_data[ivs]
    observations = experiment_data[dvs]

    # split into train and test datasets
    conditions_train, conditions_test, observations_train, observations_test = train_test_split(conditions,
                                                                                                observations)

    print("#### EXPERIMENT CONDITIONS (X):")
    print(conditions)
    print("#### EXPERIMENT OBSERVATIONS (Y):")
    print(observations)

    # fit theorist
    theorist.fit(conditions_train, observations_train)

    # compute prediction for validation set
    predictions = theorist.predict(conditions_test)

    # evaluate theorist performance
    error = (predictions - observations_test).pow(2)
    error = error.mean()

    print("#### IDENTIFIED EQUATION:")
    print(theorist.print_eqn())

    print("#### VALIDATION SET MSE:")
    print(error)

    experiment_runner.plotter(model=theorist)

# Demonstration of the Theorist's fitting, predicting and obtaining capabilities
The ParabolaRegression class fits a parabolic function to the data. The parabolic function is defined as:

$$
y = a \times x^2 + b \times x + c
$$

where $a$, $b$, and $c$ are the coefficients to be estimated. This model captures the quadratic relationship between the independent and dependent variables, allowing for the identification of non-linear patterns in the data.

In [ ]:
my_theorist = ParabolaRegression()

## Demonstration of the Theorist's recovery of two ground-truth models

### Steven's Power Law

Steven's power law describes the relationship between a stimulus's intensity $S$ ($range: [0.01, 5.00]$) and its perceived magnitude $y$. According to this law, humans are less sensitive to changes in high-intensity stimuli compared to low-intensity ones, leading to a power-law relationship between stimulus intensity and perceived magnitude:


$\text{perceived intensity} = {S}^\alpha$

where $\alpha = 0.80$, resulting in diminishing effects of increases in stimulus intensity.

In [ ]:
benchmark(experiment_runner=stevens_power_law(), theorist=my_theorist)

### Weber-Fechner-Law

The Weber-Fechner law quantifies the minimum change in a stimulus required to be noticeable. Similar to Steven's power law, the greater the intensity of a stimulus, the larger the change needed to be perceivable. This relationship is hypothesized to be proportional to the logarithm of the ratio between the two stimuli:

$\text{perceived intensity} = \log\left(\dfrac{S_1}{S_2}\right)$


where $S_1$ ($range: [0.01, 5.00]$) is the intensity of a physical stimulus (e.g., the luminosity of a lamp), $S_2$ ($range: [0.01, 5.00]$ ) is a reference stimulus (e.g., the luminosity of a background light), and $y$ is the perceived stimulus intensity (e.g. the perception of the lamp's luminosity).

In [ ]:
benchmark(experiment_runner=weber_fechner_law(), theorist=my_theorist)

### Expected Utility Model with Two Choice Options

The expected utility model evaluates decision-making under uncertainty, quantifying the expected value of different choices based on their potential outcomes and associated probabilities. The model assumes that individuals aim to maximize their expected utility when faced with two options. Each option has a specific value and probability, influenced by a certain level of noise.

For two choice options, the expected value of each option is calculated as follows:

$$
E_A = V_A \times P_A
$$

$$
E_B = V_B \times P_B
$$

where:
- $V_A$ and $V_B$ represent the values of options A and B respectively.
- $P_A$ and $P_B$ represent the probabilities associated with these options.

The probability of choosing option A $P_{\text{choose}_A}$ is then determined using the softmax function, which considers the expected values of both options and a choice temperature parameter that influences the sensitivity to differences in expected values:

$$
P_{\text{choose}_A} = \frac{\exp(E_A / \beta)}{\exp(E_A / \beta) + \exp(E_B / \beta)}
$$

In this model:
- $\beta$ controls the degree of randomness in the choice, with higher values leading to more exploration and lower values leading to more deterministic choices based on the expected values.
- The softmax function ensures that the probabilities sum to 1, providing a normalized measure of the likelihood of choosing each option.

This model captures the influence of value, probability, and noise on decision-making, reflecting the complexity and variability of human choices under uncertainty.

In [ ]:
benchmark(experiment_runner=expected_value_theory(), theorist=my_theorist)